## Config

In [55]:
# Import dependencies
import tekore as tk
import os
import time
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

In [56]:
### Authenticate Tekore and Authorize App
# Get client token
app_token = tk.request_client_token(CLIENTID, CLIENTSECRET)
# Get a user token; Note the need to ask for write scope in order to create/edit playlists
# This will open a browser window; the user will need to copy the URL from the browser and paste it into the VSCode Command Palette
user_token = tk.prompt_for_user_token(CLIENTID, CLIENTSECRET, REDIRECT_URI, scope=tk.scope.write)
# Create spotify instance
spotify = tk.Spotify(app_token)
# Get the user's spotifyID; The spotify instance will have to reference the user_token to be authorized for this data
with spotify.token_as(user_token):
    userID = spotify.current_user().id

Opening browser for Spotify login...


## Create an empty playlist in the user's account

In [57]:
# Create a playlist for the user
with spotify.token_as(user_token):
    newPlaylist = spotify.playlist_create(userID, "Summer Playlist Artist Candidates", public=False, description='Top three songs from each artist playing a summer festival this year (<5 years old)')
playlistID = newPlaylist.id

## Add some songs to the playlist

In [58]:
# Extract list of songs
songs = pd.read_csv('songs1.csv')
songs

,artistName,artistID,songURI,artistPop,artistGenre
0,Taylor Swift,06HL4z0CvFAxyc27GXpf02,spotify:track:0V3wPSX9ygBnCm8psDIegu,100.0,pop
1,Taylor Swift,06HL4z0CvFAxyc27GXpf02,spotify:track:3rWDp9tBPQR9z6U5YyRSK4,100.0,pop
2,Taylor Swift,06HL4z0CvFAxyc27GXpf02,spotify:track:5jQI2r1RdgtuT8S3iG8zFC,100.0,pop
3,Bad Bunny,4q3ewBCX7sLwd24euuV69X,spotify:track:38UYeBLfvpnDSG9GznZdnL,99.0,"reggaeton, trap latino, urbano latino"
4,Bad Bunny,4q3ewBCX7sLwd24euuV69X,spotify:track:6Sq7ltF9Qa7SNFBsV5Cogx,99.0,"reggaeton, trap latino, urbano latino"
...,...,...,...,...,...
2612,Dj Diesel,6TxBySn4sDWKZuSR5gpp6H,spotify:track:1lJUFUq9WBk7OHcxxodyZ4,19.0,NaN
2613,Clockwork,2h27SMMeQd7VEcbVwfu71E,spotify:track:2axkjRxMWykrM0k7ipO6om,14.0,NaN
2614,Clockwork,2h27SMMeQd7VEcbVwfu71E,spotify:track:3DOwkoLAlChRPiepG6Vgif,14.0,NaN
2615,Clockwork,2h27SMMeQd7VEcbVwfu71E,spotify:track:2sNuo9Q0jqYxnaZ3OBLecF,14.0,NaN


In [59]:
# Cut the df into chunks
n = 0
while n < len(songs):
    songsToAdd = songs.iloc[n:(n+100)]
    # Add songs to the playlist
    with spotify.token_as(user_token):
        additions = spotify.playlist_add(playlistID, list(songsToAdd['songURI']), position=None)
    n += 100
    # Wait 2 seconds so we do not exceed our API call allowance
    time.sleep(2)
